In [2]:
from src.dataset import ConnTextULDataset
import pandas as pd
import torch as pt
import glob
import json
import tqdm

%reload_ext autoreload
%autoreload 2

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [3]:
# Load in a dataset to access Matt's Traindata class which
# is embedded inside the phonology tokenizer (consider changing this
from pathlib import Path

config = type(
    "config",
    (object,),
    {"dataset_filename": Path("/workspaces/ConnTextUL/data/data.csv")},
)
ds = ConnTextULDataset(config)

Cache folder: /workspaces/ConnTextUL/data/.cache already exists


In [3]:
# Read in the Woodcock Johnson III Form A dataset for assessment
with open("data/wj_iii_form_a.json", "r") as f:
    wj3_json = json.load(f)

In [4]:
# Create a template dataframe to populate for each checkpoint
df = pd.DataFrame(
    columns=[
        "word_raw",
        "phon_target",
        "phon_prediction",
        "correct",
        "phon_target_features",
        "phon_prediction_features",
        "phon_prediction_probabilities",
        "global_encoding",
        "in_wj3",
        "in_traindata",
    ]
)

In [35]:
# read in the checkpoint file names. We will load one at a time
# and generate predictions of the wj3 assessments one at at ime
checkpoints = glob.glob("models/root*.pth")
checkpoints.sort()
print(f"{checkpoints=}")

checkpoints=['models/root_2024-03-04_14h08m31745ms_chkpt001.pth', 'models/root_2024-03-04_14h08m31745ms_chkpt002.pth', 'models/root_2024-03-04_16h02m25184ms_chkpt001.pth', 'models/root_2024-03-04_16h02m25184ms_chkpt002.pth', 'models/root_2024-03-04_16h07m30392ms_chkpt001.pth', 'models/root_2024-03-04_16h07m30392ms_chkpt002.pth', 'models/root_2024-03-04_16h08m15959ms_chkpt001.pth', 'models/root_2024-03-04_16h08m15959ms_chkpt002.pth', 'models/root_2024-03-04_17h17m55970ms_chkpt001.pth', 'models/root_2024-03-04_17h17m55970ms_chkpt002.pth']


In [210]:
from src.model import Model
from addict import Dict as AttrDict

chkpt = pt.load(checkpoints[-1])
model = Model(AttrDict(chkpt["config"]), ds)
model.load_state_dict(chkpt["model_state_dict"])

datum = ds.character_tokenizer.encode(["hello", "a", "jingleheimer"])
pred = model.generate(
    "o2p",
    datum["enc_input_ids"],
    datum["enc_pad_mask"],
    datum["enc_input_ids"],
    datum["enc_pad_mask"],
    deterministic=True,
)
print(f"{pred=}")
for k, v in pred.items():
    print(f"{k=}, {v=}")

--In Generate--
orth_enc_input:  torch.Size([3, 14])
orth_enc_pad_mask:  torch.Size([3, 14])
phon_enc_input:  torch.Size([3, 14])
phon_enc_pad_mask:  torch.Size([3, 14])
generated_phon_tokens=[[tensor([31])], [tensor([31])], [tensor([31])]]
--In phonology_decoder_loop--
generated_phon_embeddings.shape=torch.Size([3, 1, 64])
prompt_encoding.shape=torch.Size([3, 1, 64])
len(generated_phon_tokens)=3
	generated_phon_embeddings:  torch.Size([3, 1, 64])
	prompt_encoding:  torch.Size([3, 1, 64])
	step_mask:  torch.Size([1, 1])
--In phono_sample--
last_token_probs.shape=torch.Size([3, 2, 33])
last_token_probs=tensor([[[0.8323, 0.7345, 0.6179, 0.8690, 0.7797, 0.7236, 0.6461, 0.6367,
          0.7406, 0.6195, 0.8935, 0.7721, 0.6925, 0.6723, 0.6889, 0.5953,
          0.8798, 0.7288, 0.8581, 0.7074, 0.6787, 0.5445, 0.6221, 0.6496,
          0.6130, 0.6879, 0.8145, 0.6857, 0.8033, 0.5670, 0.6682, 0.5927,
          0.6022],
         [0.1677, 0.2655, 0.3821, 0.1310, 0.2203, 0.2764, 0.3539, 0.3633,
  

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list

In [127]:
t = pt.softmax(pt.randn((3,2,4)),1); t

tensor([[[0.4743, 0.6130, 0.4738, 0.3406],
         [0.5257, 0.3870, 0.5262, 0.6594]],

        [[0.3976, 0.7930, 0.6806, 0.3008],
         [0.6024, 0.2070, 0.3194, 0.6992]],

        [[0.1859, 0.0626, 0.2934, 0.6691],
         [0.8141, 0.9374, 0.7066, 0.3309]]])

In [195]:
[[pt.tensor([31])]]*2

[[tensor([31])], [tensor([31])]]

In [131]:
for t_i in t:
    print(t_i[1])

tensor([0.5257, 0.3870, 0.5262, 0.6594])
tensor([0.6024, 0.2070, 0.3194, 0.6992])
tensor([0.8141, 0.9374, 0.7066, 0.3309])


In [83]:
t[:,1,:]

tensor([[0.5280, 0.0858, 0.4528, 0.4649],
        [0.3310, 0.6017, 0.5014, 0.5309],
        [0.4542, 0.5266, 0.6082, 0.2605]])

In [133]:
fp = (t[:,1,:] > 0.5).long(); fp

tensor([[1, 0, 1, 1],
        [1, 0, 0, 1],
        [1, 1, 1, 0]])

In [165]:
fp = pt.bernoulli(t[:,1,:]).long(); fp.shape

torch.Size([3, 4])

In [157]:
indices = pt.where(fp); indices

(tensor([0, 0, 1, 1, 2, 2, 2]), tensor([0, 3, 0, 3, 0, 1, 2]))

In [164]:
for x, y in zip(*indices):
    print(x.item(), y.item())

0 0
0 3
1 0
1 3
2 0
2 1
2 2


In [54]:
len(pred["phon_probs"])

13

In [53]:
for op, p in zip(old_pred, pred):
    if op == p:
        print("True")
    else:
        print("False")

True
True
True
True
True
True


In [51]:
old_pred = pred

In [ ]:
ortho_sample = True

In [6]:
# Here we take all orthographic words from both WJ3 and the
# original dataset and remove duplicates.
all_words = set(wj3_json.keys())
all_words.update(ds.words)  # Take the union of both sets

In [ ]:
writer = pd.ExcelWriter("wj3_predictions.xlsx", engine="openpyxl")
for checkpoint in tqdm.tqdm(checkpoints):
    # Empty the dataframe to prepare for the next checkpoint (aka excel page)
    df.drop(df.index, inplace=True)
    chkpt = pt.load(checkpoint)
    model = chkpt["model"]
    # In this loop, orth is the orthographic form of the target word and
    # phon is the phonological form of the target word (not phonetic vectors)
    for orth in all_words:
        new_row = {}
        # Since all_words is a union of both WJ3 and the original dataset,
        # the phonological form of any orth in all_words is in either WJ3 or
        # the original cmudict dataset. We check both below
        in_wj3 = orth in wj3_json.keys()
        new_row["in_wj3"] = in_wj3
        in_traindata = orth in ds.words
        new_row["in_traindata"] = in_traindata
        if in_traindata:
            phon = ds.cmudict[orth]
        else:
            phon = wj3_json[orth]

        datum = ds.character_tokenizer.encode(orth)
        pred = chkpt["model"].generate(
            "o2p",
            datum["enc_input_ids"],
            datum["enc_pad_mask"],
            datum["enc_input_ids"],
            datum["enc_pad_mask"],
            deterministic=True,
        )
        # Save the original input orthography
        new_row["word_raw"] = orth
        # Save the target phonology for the above input orthography
        new_row["phon_target"] = ":".join(phon)
        # Remove the start and end tokens from each phonological vector
        # and convert them from tensors to lists
        phon_pred_features = [tensor.tolist() for tensor in pred["phon_tokens"][1:-1]]
        # Convert the phonological vectors to phonemes using Matt's handy dandy routine
        phon_pred = ds.phonology_tokenizer.traindata.convert_numeric_prediction(
            phon_pred_features, phonology=True, hot_nodes=True
        )
        # Save the model's predicted pronunciation for this word. Phonemes are
        # separated by colons
        phon_pred = ["None" if p == None else p for p in phon_pred]
        new_row["phon_prediction"] = ":".join(phon_pred)
        # Save a boolean indicating whether the prediction was correct
        new_row["correct"] = new_row["phon_target"] == new_row["phon_prediction"]
        # Save the phonological features for the target phonology
        phon_target_features = ds.phonology_tokenizer.encode([orth])
        if phon_target_features:
            phon_target_features = ";".join(
                [
                    ":".join([str(v.item()) for v in vector])
                    for vector in phon_target_features["targets"][0]
                ]
            )
        else:
            phon_target_features = "None"
        new_row["phon_target_features"] = phon_target_features
        # Save the phonological features for the predicted phonology
        phon_prediction_features = ";".join(
            [
                ":".join([str(int(v.item())) for v in vector])
                for vector in pred["phon_vecs"][1:-1]
            ]
        )
        new_row["phon_prediction_features"] = phon_prediction_features
        # Save the phonological probabilities for the predicted phonology
        phon_prediction_probabilities = ";".join(
            [
                ":".join([str(v.item()) for v in vector])
                for vector in pred["phon_probs"][1:-1]
            ]
        )
        new_row["phon_prediction_probabilities"] = phon_prediction_probabilities
        # Save the global encoding vector for the predicted phonology
        global_encoding = ":".join(
            [str(v.item()) for v in pred["global_encoding"].squeeze()]
        )
        new_row["global_encoding"] = global_encoding
        new_row = pd.Series(new_row)
        new_row = new_row.to_frame().transpose()
        df = pd.concat([df, new_row])
    df.to_excel(writer, sheet_name="epoch " + checkpoint[-7:-4])
writer.book.save("wj3_predictions.xlsx")

In [1]:
a = [{}]

In [2]:
a[0] = {"a": 1, "b": 2}

In [3]:
a

[{'a': 1, 'b': 2}]

In [4]:
a[0] = {"a": 3, "b": 4}

In [5]:
a

[{'a': 3, 'b': 4}]

In [1]:
import torch as pt

model = pt.load("models/root_2024-02-17_21h49m56470ms_chkpt001.pth")

In [2]:
model.keys()

dict_keys(['model_path', 'config', 'model', 'optimizer', 'model_id'])

In [4]:
model["config"]

{'device': 'cpu',
 'project': 'nathan',
 'num_epochs': 2,
 'batch_size_train': 8,
 'batch_size_val': 8,
 'num_layers': 2,
 'learning_rate': 0.001,
 'continue_training': False,
 'model_chkpt': '',
 'd_model': 16,
 'nhead': 2,
 'wandb': False,
 'test': True,
 'max_nb_steps': -1,
 'train_test_split': 0.9,
 'which_dataset': 100,
 'sweep': '',
 'd_embedding': 2,
 'seed': 1337,
 'nb_samples': 1000,
 'model_path': './models',
 'pathway': 'op2op',
 'save_every': 2,
 'model_id': 'root_2024-02-17_21h49m56470ms',
 'chkpt_file_exists': False,
 'epochs_completed': 0,
 'model_file_name': 'root_2024-02-17_21h49m56470ms_chkpt000.pth',
 'n_steps_per_epoch': 11}

In [12]:
set(["a", "b"]) == set(["b", "a"])

True

In [15]:
filename = None
with open(filename, "r") as f:
    wj3_json = json.load(f)

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [8]:
from pathlib import Path

In [10]:
project_root = Path("/workspaces/ConnTextUL/")

In [11]:
project_root

PosixPath('/workspaces/ConnTextUL')

In [13]:
project_root.parent.parent

PosixPath('/')

In [122]:
a = [[] for _ in range(3)]

In [123]:
a[0].append(1)

In [124]:
a

[[1], [], []]